In [1]:
from getsparkcontext import sc

In [5]:
base_rdd = sc.textFile('file:///Users/hdagar3/Documents/Spark_Things/Spark_Course_Files/Marvel-Graph.txt')
# this textFile() function creates one line as one element of rdd, and definition of one line means untill it gets '\n'
# character

In [6]:
print(base_rdd)

file:///Users/hdagar3/Documents/Spark_Things/Spark_Course_Files/Marvel-Graph.txt MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:0


In [7]:
print(base_rdd.count())  # no of lines or no of elements in rdd

6589


In [13]:
print(base_rdd.collect())

In [9]:
def split_function(line):
    info = line.split()
    superhero_id = int(info[0])
    no_of_friends = len(info)-1
    return (superhero_id,no_of_friends)

In [21]:
superheroid_friends_rdd = base_rdd.map(split_function)

In [26]:
# print(superheroid_friends_rdd.collect())

In [64]:
reduced_superheroid_rdd = superheroid_friends_rdd.reduceByKey(lambda x,y:x+y)  
# because it might happen that superhero_id is on multiple lines

print(reduced_superheroid_rdd.collect())

In [54]:
reversed_rdd = reduced_superheroid_rdd.map(lambda x:(x[1],x[0]))

In [55]:
print(reversed_rdd.count())

6486


In [59]:
string = r'234    "Marver name is this\and lot much"'  # raw string means don't process the string with slahes

In [60]:
print(string.split("\""))

['234    ', 'Marver name is this\\and lot much', '']


In [61]:
print(int(string.split("\"")[0]))

234


In [63]:
f = lambda (x,y):x+y   # valid in python2

SyntaxError: invalid syntax (<ipython-input-63-8f941a65aa91>, line 1)

In [84]:
max_tuple = reversed_rdd.max() # max() & min() are actions , sorting is done based on first element of tuple 

In [85]:
print(type(max_tuple))

<class 'tuple'>


In [86]:
print(max_tuple)  # (no_of_friends,superhero_id)

(1933, 859)


In [87]:
# Now I need to go through the marvel-names file to fetch the name of marvel corresponding to 856 marvel id
# I could make a dict object and then broadcast it to very node but in order to contruct a dict, it should accomodate
# inside RAM. If file is too huge then that concept of making dict would fail and then broadcase could not be done if
# file is too huge

In [88]:
# So lets construct rdd of marvel-names file also

In [89]:
marvel_names_rdd = sc.textFile('file:///Users/hdagar3/Documents/Spark_Things/Spark_Course_Files/Marvel-Names.txt')

In [104]:
def split_marvel_names(line):
    info = line.split("\"")
    marvel_id = int(info[0].strip())
    marvel_name = info[1].encode("utf-8")   # encoding is your choice
    return (marvel_id,marvel_name)

In [105]:
processed_marvel_name_rdd = marvel_names_rdd.map(split_marvel_names)
# print(processed_marvel_name_rdd.collect())

In [106]:
list_of_values = processed_marvel_name_rdd.lookup(max_tuple[1])  # lookup() is action

In [107]:
print(list_of_values)

[b'CAPTAIN AMERICA']


In [108]:
print(list_of_values[0])

b'CAPTAIN AMERICA'


In [ ]:
# END